In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/digit-recognizer/train.csv
/kaggle/input/digit-recognizer/test.csv
/kaggle/input/digit-recognizer/sample_submission.csv


In [2]:
import pandas as pd
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.


In [3]:
df =  pd.read_csv("/kaggle/input/digit-recognizer/train.csv")

In [4]:
df.shape

(42000, 785)

In [5]:
test= pd.read_csv("/kaggle/input/digit-recognizer/test.csv")

In [6]:
df.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
x = df.iloc[:,1:].values
y= df.iloc[:,0].values

In [8]:
from sklearn.model_selection import train_test_split 
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.1)

In [9]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)


In [10]:
from keras.utils import np_utils
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)

In [11]:
model = Sequential()
model.add(Dense(units=784,activation='relu',input_dim = 784))
from keras.layers import Dropout

In [12]:
model.add(Dense(units = 128, activation = 'relu'))
model.add(Dense(units = 128, activation = 'relu'))
model.add(Dropout(0.2))
model.add(Dense(units = 64, activation = 'relu'))
model.add(Dense(units = 64, activation = 'relu'))
model.add(Dropout(0.2))
model.add(Dense(units = 32, activation = 'relu'))
model.add(Dense(units = 32, activation = 'relu'))
model.add(Dropout(0.05))

In [13]:
model.add(Dense(units = 10, activation = 'softmax'))

In [14]:
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [15]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 784)               615440    
_________________________________________________________________
dense_2 (Dense)              (None, 128)               100480    
_________________________________________________________________
dense_3 (Dense)              (None, 128)               16512     
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_5 (Dense)              (None, 64)                4160      
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)               

In [16]:
model.fit(X_train, y_train, validation_data = (X_test, y_test), batch_size = 28, epochs = 10)

Train on 37800 samples, validate on 4200 samples
Epoch 1/10
37800/37800 [==============================] - 15s 398us/step - loss: 0.4257 - accuracy: 0.8789 - val_loss: 0.2029 - val_accuracy: 0.9450
Epoch 2/10
37800/37800 [==============================] - 15s 401us/step - loss: 0.2042 - accuracy: 0.9480 - val_loss: 0.1693 - val_accuracy: 0.9540
Epoch 3/10
37800/37800 [==============================] - 15s 397us/step - loss: 0.1468 - accuracy: 0.9636 - val_loss: 0.1547 - val_accuracy: 0.9631
Epoch 4/10
37800/37800 [==============================] - 15s 388us/step - loss: 0.1186 - accuracy: 0.9699 - val_loss: 0.1570 - val_accuracy: 0.9612
Epoch 5/10
37800/37800 [==============================] - 15s 388us/step - loss: 0.0996 - accuracy: 0.9751 - val_loss: 0.1729 - val_accuracy: 0.9571
Epoch 6/10
37800/37800 [==============================] - 15s 390us/step - loss: 0.0893 - accuracy: 0.9787 - val_loss: 0.1468 - val_accuracy: 0.9662
Epoch 7/10
37800/37800 [==============================] -

In [17]:
acc = model.evaluate(X_test, y_test)
acc

4200/4200 [==============================] - 0s 64us/step


[0.14513553762108564, 0.9714285731315613]

In [18]:
test_x = test.iloc[:,:].values

In [19]:
test_pred = model.predict(test_x)

In [20]:

results = test_pred.argmax(axis=1)

In [21]:
results

array([2, 0, 9, ..., 3, 9, 2])

In [22]:
results = pd.Series(results,name="Label")
submission = pd.concat([pd.Series(range(1,28001),name = "ImageId"),results],axis = 1)
submission.to_csv("submission.csv",index=False)